![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
df = pd.read_csv('rental_info.csv')
print(df.head())

                 rental_date                return_date  amount  release_year  \
0  2005-05-25 02:54:33+00:00  2005-05-28 23:40:33+00:00    2.99        2005.0   
1  2005-06-15 23:19:16+00:00  2005-06-18 19:24:16+00:00    2.99        2005.0   
2  2005-07-10 04:27:45+00:00  2005-07-17 10:11:45+00:00    2.99        2005.0   
3  2005-07-31 12:06:41+00:00  2005-08-02 14:30:41+00:00    2.99        2005.0   
4  2005-08-19 12:30:04+00:00  2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_

In [2]:
SEED = 9
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


Creating a column named **"rental_length_days"** using the columns **"return_date"** and **"rental_date"**, and adding it to the pandas DataFrame. This column contains information on how many days a DVD has been rented by a customer.

In [3]:
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['rental_length'] = (df['return_date'] - df['rental_date']).dt.days
df.drop(['return_date', 'rental_date'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   amount            15861 non-null  float64
 1   release_year      15861 non-null  float64
 2   rental_rate       15861 non-null  float64
 3   length            15861 non-null  float64
 4   replacement_cost  15861 non-null  float64
 5   special_features  15861 non-null  object 
 6   NC-17             15861 non-null  int64  
 7   PG                15861 non-null  int64  
 8   PG-13             15861 non-null  int64  
 9   R                 15861 non-null  int64  
 10  amount_2          15861 non-null  float64
 11  length_2          15861 non-null  float64
 12  rental_rate_2     15861 non-null  float64
 13  rental_length     15861 non-null  int64  
dtypes: float64(8), int64(5), object(1)
memory usage: 1.7+ MB


In [4]:
df['special_features'].value_counts()

special_features
{Trailers,Commentaries,"Behind the Scenes"}                     1308
{Trailers}                                                      1139
{Trailers,Commentaries}                                         1129
{Trailers,"Behind the Scenes"}                                  1122
{"Behind the Scenes"}                                           1108
{Commentaries,"Deleted Scenes","Behind the Scenes"}             1101
{Commentaries}                                                  1089
{Commentaries,"Behind the Scenes"}                              1078
{Trailers,"Deleted Scenes"}                                     1047
{"Deleted Scenes","Behind the Scenes"}                          1035
{"Deleted Scenes"}                                              1023
{Commentaries,"Deleted Scenes"}                                 1011
{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}     983
{Trailers,Commentaries,"Deleted Scenes"}                         916
{Trailers,"Delete

Creating two columns of dummy variables from __"special_features"__, which takes the value of 1 when:
The value is __"Deleted Scenes"__, storing as a column called __"deleted_scenes"__.
The value is __"Behind the Scenes"__, storing as a column called __"behind_the_scenes"__.
We then drop the __"special features"__ column.

In [5]:
df["deleted_scenes"] =  np.where(df["special_features"].str.contains("Deleted Scenes"), 1,0)
df["behind_the_scenes"] =  np.where(df["special_features"].str.contains("Behind the Scenes"), 1,0)
df.drop('special_features', axis=1, inplace=True)

In [6]:
df.describe()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length,deleted_scenes,behind_the_scenes
count,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000
mean,4.217161,2006.885379,2.944101,114.994578,20.224727,0.204842,0.200303,0.223378,0.198726,23.355504,14832.841876,11.389287,4.525944,0.497320,0.536347
std,2.360383,2.025027,1.649766,40.114715,6.083784,0.403599,0.400239,0.416523,0.399054,23.503164,9393.431996,10.005293,2.635108,0.500009,0.498693
min,0.990000,2004.000000,0.990000,46.000000,9.990000,0.000000,0.000000,0.000000,0.000000,0.980100,2116.000000,0.980100,0.000000,0.000000,0.000000
25%,2.990000,2005.000000,0.990000,81.000000,14.990000,0.000000,0.000000,0.000000,0.000000,8.940100,6561.000000,0.980100,2.000000,0.000000,0.000000
50%,3.990000,2007.000000,2.990000,114.000000,20.990000,0.000000,0.000000,0.000000,0.000000,15.920100,12996.000000,8.940100,5.000000,0.000000,1.000000
75%,4.990000,2009.000000,4.990000,148.000000,25.990000,0.000000,0.000000,0.000000,0.000000,24.900100,21904.000000,24.900100,7.000000,1.000000,1.000000
max,11.990000,2010.000000,4.990000,185.000000,29.990000,1.000000,1.000000,1.000000,1.000000,143.760100,34225.000000,24.900100,9.000000,1.000000,1.000000


Check if we have any missing values. 

In [7]:
df.isna().sum()

amount               0
release_year         0
rental_rate          0
length               0
replacement_cost     0
NC-17                0
PG                   0
PG-13                0
R                    0
amount_2             0
length_2             0
rental_rate_2        0
rental_length        0
deleted_scenes       0
behind_the_scenes    0
dtype: int64

Making a pandas DataFrame called X containing all the appropriate features used to run the regression models, avoiding columns that leak data about the target.
Choose the "rental_length_days" as the target column and save it as a pandas Series called y.

In [8]:
X= df.drop('rental_length', axis=1).values
y= df['rental_length'].values
type(X)

numpy.ndarray

Split the data into __X_train, y_train, X_test, and y_test train and test sets__, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
print(X_train.shape, X_test.shape)

(12688, 14) (3173, 14)


Identify the features with the best prediction power for the target variable.
Using __Lasso()__ from sklearn.linear_model allows us to look at feature importance by accessing the model's .coef_ attribute, where values over 0 indicate a contribution to the model's performance.

In [10]:
from sklearn.linear_model import Lasso
lasso = Lasso(random_state=SEED, alpha=0.5)
lasso.fit(X_train, y_train)

mask = lasso.coef_ > 0

# Apply the same mask to both train and test sets
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

print(X_train_selected.shape)  # Check the shape after masking
print(X_test_selected.shape)

(12688, 3)
(3173, 3)


Try a variety of regression models.
The __RandomizedSearchCV()__ function allows us to search for the best model performance using random values from ranges of hyperparameters.
The mean squared error is computed and the best model i.e. the model with the __lowest MSE__ is selected.

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

models = {
    'LinearRegression': LinearRegression(),
    'DecisionTreeRegressor': DecisionTreeRegressor(random_state=SEED),
    'RandomForestRegressor': RandomForestRegressor(random_state=SEED)
}

# Define hyperparameter grids for RandomizedSearchCV
param_grids = {
    'DecisionTreeRegressor': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': np.arange(2, 10),
        'min_samples_leaf': np.arange(1, 5),
        'max_features': ['sqrt', 'log2', None]
    },
    'RandomForestRegressor': {
        'n_estimators': np.arange(10, 300, 10),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': np.arange(2, 10),
        'min_samples_leaf': np.arange(1, 5),
        'max_features': ['sqrt', 'log2', None]
    }
}

best_model = None
best_mse = float('inf') 
# Loop through models and tune the tree-based ones
for model_name, model in models.items():
    if model_name in param_grids:  # If the model has hyperparameters to tune
        print(f"Tuning hyperparameters for {model_name}...")
        search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grids[model_name],
            n_iter=10,  
            scoring='neg_mean_squared_error',
            cv=5,  
            random_state=SEED,
            n_jobs=-1
        )
        search.fit(X_train, y_train)
        best_model = search.best_estimator_
    else:
        # If no hyperparameters to tune (LinearRegression)
        best_model = model
        best_model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{model_name} Mean Squared Error: {mse}")
    # Check if this model has the lowest MSE
    if mse < best_mse:
        best_mse = mse
        best_model = best_model  # Save the best model

LinearRegression Mean Squared Error: 2.941723864697651
Tuning hyperparameters for DecisionTreeRegressor...
DecisionTreeRegressor Mean Squared Error: 2.2316908200093875
Tuning hyperparameters for RandomForestRegressor...
RandomForestRegressor Mean Squared Error: 2.020842211224755
